In [298]:
import pandas as pd
import random

In [299]:
PATH_TRAIN = "C:/Users/lilir/Desktop/natural language processing/project/project/train.bio"

In [300]:
def read_iob2_file(path):
    data = []
    current_words = []
    current_tags = []

    for line in open(path, encoding='utf-8'):
        line = line.strip()

        if line:
            if line[0] == '#':
                continue
            tok = line.split('\t')

            current_words.append(tok[1])
            current_tags.append(tok[2])
        else:
            if current_words:
                data.append((current_words, current_tags))
            current_words = []
            current_tags = []

    if current_tags != []:
        data.append((current_words, current_tags))

    df = pd.DataFrame(data, columns=['words', 'tags'])
    df['id'] = df.index
    df = df[['id', 'words', 'tags']]
    
    return df

In [301]:
def read_bio_file(path):
    
    data = []
    current_words = []
    current_tags = []

    with open(path, "r", encoding="utf-8") as f:
        lines = f.readlines()[2:]
        
    for line in lines:
        
        line = line.strip()
        
        if line: # if line is not an empty line
            tok = line.split('\t')
            current_words.append(tok[0])
            current_tags.append(tok[3])
            
        else:
            if current_words:
                data.append((current_words, current_tags))
            current_words = []
            current_tags = []
            
            
    if current_tags != []:
        data.append((current_words, current_tags))

    df = pd.DataFrame(data, columns=['words', 'tags'])
    df['id'] = df.index
    df = df[['id', 'words', 'tags']]
    
    return df

In [368]:
data = read_bio_file(PATH_TRAIN)
sentences = data["words"]
total_num_of_sentences = len(sentences)
#sentences.head()
#sentences[1]
#data.head()

In [372]:
def deletion(data, percent_mod_sentences):
    #number of sentences modified (percentage-wise)
    num_mod_sent = (percent_mod_sentences*total_num_of_sentences)//100

    #random decision of the type of mistake made
    sentences_list = list(sentences)
    mod_wordlist = []                   # list of modified words 
    mod_sent_ids = []                   # list of modified sentence ids
    mod_word_id_in_sent = []            # list of ids of modified words inside the sentences 

    type_of_mistake = random.randint(1, 3)
    print(type_of_mistake)

    #mistake type 1: missed last letter
    if type_of_mistake == 1:
        while len(mod_wordlist) < num_mod_sent:
            rsent = random.choice(sentences)
            rwordIdx = random.randint(0, len(rsent) - 1)
            
            rword = rsent[rwordIdx]
            if len(rword) > 1:
                mod_wordlist.append(rword[:-1])
                mod_word_id_in_sent.append(rwordIdx)
                mod_sent_ids.append(sentences_list.index(rsent))
    
    #mistake type 2: missed first letter
    elif type_of_mistake == 2:    
        while len(mod_wordlist) < num_mod_sent:
            rsent = random.choice(sentences)
            rwordIdx = random.randint(0, len(rsent) - 1)
            rword = rsent[rwordIdx]
            if len(rword) > 1:
                mod_wordlist.append(rword[1:])
                mod_word_id_in_sent.append(rwordIdx)
                mod_sent_ids.append(sentences_list.index(rsent))


    #mistake type 3: missed random letter
    elif type_of_mistake == 3:
        while len(mod_wordlist) < num_mod_sent:
            rsent = random.choice(sentences)
            rwordIdx = random.randint(0, len(rsent) - 1)
            rword = rsent[rwordIdx]
            if len(rword) >= 3:
                rIdx = random.randint(0, len(rword) - 1)
                modWord = rword[:rIdx] + rword[rIdx+1:]
                mod_wordlist.append(modWord)
                mod_sent_ids.append(sentences_list.index(rsent))
                mod_word_id_in_sent.append(rwordIdx)

    #returns a list of modified words, the ids of the modified sentences, the position of the modified word in its respective sentence
    return mod_wordlist, mod_sent_ids, mod_word_id_in_sent

#test
typos, sentence_idx, words_sent_ids = deletion(sentences, 0.5)
print(typos, sentence_idx, words_sent_ids)


2
['o', 'orse', 'head', 'icking', 'ourself', 'et', 'pgrade', 'iss'] [1382, 1501, 919, 25, 742, 1567, 1318, 639] [8, 10, 7, 6, 16, 8, 30, 4]


In [373]:
data['words'][1382][8]

'to'

In [375]:
# function that overwrites the dataframe with the sentences, which contain typos
def modify_df(dataframe, modified_sentence_ids, modified_word_idx, typos):
    for i in range(len(typos)):
        dataframe['words'][modified_sentence_ids[i]][modified_word_idx[i]] = typos[i]
    
    return dataframe

# test
modified_df = modify_df(data, sentence_idx, words_sent_ids, typos)
modified_df['words'][1382][8]


'o'